# Real-time demo

One of the strengths of DSINE is that it runs in real-time. 

Here we explain how you can build your own real-time demo.

In [1]:
import cv2
import torch 

import sys
sys.path.append('../')

from utils.demo_data import define_input

`InputStream` will load frames from some source. You can specify the source by setting `input` to the following:

* `screen`: screenshot of your screen
* `webcam`: webcam
* `rs`: a realsense camera
* `youtube`: extract frames from a Youtube video

Besides `input`, you should also provide `device` where the loaded image would be sent to.

Each option requires additional keyword arguments.

### `input = "screen"`

* For this option, you need to provide:
  * `intrins`: 
    * Camera intrinsics as a torch tensor of shape (3, 3)
    * If `None`, it will be assumed that the principal point is at the center and that the field-of-view is 60 degrees
  * `top`, `left`, `width`, `height`:
    * Section of the screen to capture

In [2]:
input = 'screen'
device = 0

kwargs = dict(
    intrins = None,
    top = (1080-480) // 2,
    left = (1920-640) // 2,
    height = 480,
    width = 640,
)

InputStream = define_input(input=input, device=device, **kwargs)

### `input = "webcam"`

* For this option, you need to provide:
  * `intrins`: 
    * Camera intrinsics as a torch tensor of shape (3, 3)
    * If `None`, it will be assumed that the principal point is at the center and that the field-of-view is 60 degrees
  * `new_width`:
    * If the webcam image resolution is too low/high, you can set `new_width` to resize it
    * By default, the image will not be resized
  * `webcam_index`:
    * OpenCV will find a video source based on the provided index
    * Normally, `0` or `1` would work

In [3]:
input = 'webcam'
device = 0

kwargs = dict(
    intrins = None,
    new_width = -1,
    webcam_index = 1,
)

InputStream = define_input(input=input, device=device, **kwargs)

### `input = "rs"` (realsense)

NOTE: install `pyrealsense2` by 

```
python -m pip install pyrealsense2
```

* For this option, you need to provide:
  * `enable_auto_exposure`: turn on/off auto exposure
  * `enable_auto_white_balance`: turn on/off auto WB

In [4]:
input = 'rs'
device = 0

kwargs = dict(
    enable_auto_exposure = True,
    enable_auto_white_balance = True,
)

InputStream = define_input(input=input, device=device, **kwargs)

### `input = "youtube"`

NOTE: Install `vidgear` and `yt_dlp` by

```
python -m pip install vidgear
python -m pip install yt_dlp
```

* For this option, you need to provide:
  * `intrins`: 
    * Camera intrinsics as a torch tensor of shape (3, 3)
    * If `None`, it will be assumed that the principal point is at the center and that the field-of-view is 60 degrees
  * `new_width`:
    * The video will be resized to have this width
  * `video_id`:
    * Youtube video id (e.g. `https://www.youtube.com/watch?v=dQw4w9WgXcQ` $\rightarrow$ `video_id="dQw4w9WgXcQ"`)

In [5]:
input = 'youtube'
device = 0

kwargs = dict(
    intrins = None,
    new_width = 1640,
    video_id = "dQw4w9WgXcQ",
)

InputStream = define_input(input=input, device=device, **kwargs)

17:33:06 ::    Helper     ::   INFO   :: Running VidGear Version: 0.3.2
17:33:06 ::    Helper     :: WARNING  :: GStreamer not found!
17:33:06 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
17:33:06 ::    CamGear    ::   INFO   :: Verifying Streaming URL using yt-dlp backend. Please wait...
17:33:09 ::    CamGear    ::   INFO   :: [Backend] :: Streaming URL is fully supported. Available Streams are: [144p, 240p, 360p, 480p, 720p, 1080p, best, worst]
17:33:09 ::    CamGear    ::  DEBUG   :: Using `best` resolution for streaming.
17:33:09 ::    CamGear    ::  DEBUG   :: YouTube source ID: `dQw4w9WgXcQ`, Title: `Rick Astley - Never Gonna Give You Up (Official Music Video)`, Quality: `best`
17:33:09 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!
17:33:10 ::    Helper     :: WARNING  :: GStreamer not found!
17:33:10 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
17:33:10 ::    CamGear    ::   INF

----

### Building a demo

Now you have all the ingredients. The code below will load the next frame and display it in some window. Replace `out` with something that you want to visualize (e.g. a network output). You can get keyboard input using `cv2.waitKey` to do some action (e.g. pause/quit the demo).

In [6]:
frame_name = 'insert your frame name here'
cv2.namedWindow(frame_name, cv2.WINDOW_NORMAL)
# uncomment below to get full-screen
# cv2.setWindowProperty(frame_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

pause = False

with torch.no_grad():
    while True:
        if pause:
                pass
        else:
            data_dict = InputStream.get_sample()
            color_image = data_dict['color_image']

            #↓↓↓↓
            #NOTE: do some operation to the img and get the output
            img = data_dict['img']
            intrins = data_dict['intrins']

            out = color_image
            #↑↑↑↑

            cv2.imshow(frame_name, out)

        # keyboard input
        k = cv2.waitKey(1)
        if k == ord(' '):
            pause = not pause
        elif k == ord('q'):
            break

    cv2.destroyWindow(frame_name)